<a href="https://colab.research.google.com/github/Autoratch/Algorithms/blob/master/verMay23-2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [54]:
import csv
import math
from datetime import datetime,timedelta
import matplotlib.pyplot as plt
import numpy as np

# default param val
platform = "MT"
forexpair = "EURUSD"
forextype = "M1"
yearmonth = "202204"
datafilepath = "/content/drive/My Drive/Colab Notebooks/forex_historical_data/DAT_"+platform+"_"+forexpair+"_"+forextype+"_"+yearmonth+".csv"
paramfilepath = "/content/drive/My Drive/Colab Notebooks/parameter.csv"
forexfilepath = "/content/drive/My Drive/Colab Notebooks/forex.csv"
bbVal = 20
sdVal = 2
rsiVal = 14
width = 5
rsiUpperLim = 70
rsiLowerLim = 30
martingale = 0

header = ["date","time","open","high","low","close"]
allData = [] # list of data
dayData = [] # list of each day containing list of data
weekData = [] # list of each week containing list of data
weekDataK = [] # list of each week containing list of data with width of K
bbDataK = [] # list of each week containing list of bb data with width of K
rsiDataK = [] # list of each week containing list of rsi data with width of K
dataIdx = [] # idx corresponding to itself
bbDataIdx = [] # idx corresponding to weekDataK
rsiDataIdx = [] # idx corresponding to weekDataK
avaDataIdx = [] # idx of available to indicate signals, combining bb and rsi; corresponding to weekDataK
dataAmount = [] # amount of data in each week
bbAmount = [] # amuount of bb in each week
rsiAmount = [] # amount of rsi in each week
signal = [] # signal for buy or sell; 1 for up, -1 for down
sigSuccess = 0 # no of success signal
sigFail = 0 # no of fail signal
sigAll = 0 # no of all signal
sigPercent = 0
totalDate = 0
totalMin = 0
totalWeek = 0

class eachData:
  def __init__(self,date,time,open,high,low,close):
    self.date = date
    self.time = time
    self.open = float(open)
    self.high = float(high)
    self.low = float(low)
    self.close = float(close)
  # def print(self):
  #   print(self.date+" "+self.time)
  #   print(self.open+" "+self.high+" "+self.low+" "+self.close)

class eachBB:
  def __init__(self,middle,upper,lower):
    self.middle = middle
    self.upper = upper
    self.lower = lower

def checkDiff(data1,data2): #check diff between data whether gap exist
  format = "%Y.%m.%d"
  datestr1 = datetime.strptime(data1.date,format)
  datestr2 = datetime.strptime(data2.date,format)
  expected = datestr1+timedelta(days=1)
  if datestr2<=expected:
    return 0 # no jump
  else:
    return 1 # gap exists

def plot(wk):
  #plot data
  data = weekDataK[wk]
  x = dataIdx[wk]
  y = []
  for i in data:
    y.append(i.open)
  plt.plot(x,y) # data open
  y = []
  for i in data:
    y.append(i.close)
  plt.plot(x,y) # data close
  #plot bb
  bb = bbDataK[wk]
  x = bbDataIdx[wk]
  y = []
  for i in x:
    y.append(bb[i].middle)
  plt.plot(x,y) # bb middle
  y = []
  for i in x:
    y.append(bb[i].upper)
  plt.plot(x,y) # bb upper
  y = []
  for i in x:
    y.append(bb[i].lower)
  plt.plot(x,y) # bb lower
  plt.show()
  #plot rsi
  rsi = rsiDataK[wk]
  x = rsiDataIdx[wk]
  y = []
  for i in x:
    y.append(rsi[i])
  plt.plot(x,y) # rsi val
  y = []
  for i in x:
    y.append(rsiUpperLim)
  plt.plot(x,y) # rsi upper
  y = []
  for i in x:
    y.append(rsiLowerLim)
  plt.plot(x,y) # rsi lower
  plt.show()

# read parameter from csv file
file = open(paramfilepath)
csvreader = csv.reader(file)
for x in csvreader:
  bbVal = int(x[0])
  sdVal = int(x[1])
  rsiVal = int(x[2])
  width = int(x[3])
  rsiUpperLim = int(x[4])
  rsiLowerLim = int(x[5])
  martingale = int(x[6])
file.close()

# read forex from csv file
file = open(forexfilepath)
csvreader = csv.reader(file)
for x in csvreader:
  forexpair = x[0]
  yearmonth = x[1]
file.close()
datafilepath = "/content/drive/My Drive/Colab Notebooks/forex_historical_data/DAT_"+platform+"_"+forexpair+"_"+forextype+"_"+yearmonth+".csv"

# read data from csv file
file = open(datafilepath)
csvreader = csv.reader(file)
for min in csvreader:
  allData.append(eachData(min[0],min[1],min[2],min[3],min[4],min[5]))
file.close()

# store and organize into days and weeks data list
totalMin = len(allData)  
tmp = ""
for min in allData:
  if min.date!=tmp:
    tmp = min.date
    totalDate+=1
for i in range(totalDate):
  dayData.append([])
tmp = ""
totalDate = 0
for min in allData:
  if min.date!=tmp:
    tmp = min.date
    totalDate+=1
  dayData[totalDate-1].append(min)
# for today in range(totalDate):
#   print(dayData[today][0].date+" : "+dayData[today][0].time+"->"+dayData[today][len(dayData[today])-1].time)
first = 1
tmp = []
for i in range(len(allData)):
  if first==1 or checkDiff(allData[i-1],allData[i])==0:
    tmp.append(allData[i])
    first = 0
  else:
    weekData.append(tmp)
    weekDataK.append([])
    bbDataK.append([])
    rsiDataK.append([])
    tmp = []
    tmp.append(allData[i])
if tmp!=[]:
  weekData.append(tmp)
  weekDataK.append([])
  rsiDataK.append([])
  bbDataK.append([])
# print(len(weekData))
totalWeek = len(weekData)
# for i in range(len(weekData)):
#   print(weekData[i][0].date+" "+weekData[i][0].time+" -> "+weekData[i][len(weekData[i])-1].date+" "+weekData[i][len(weekData[i])-1].time)

for i in range(len(weekData)):
  for j in range(len(weekData[i])):
    if j%width==width-1:
      tmp = weekData[i][j]
      tmp.open = weekData[i][j-width+1].open
      weekDataK[i].append(tmp)

# for i in range(len(weekDataK)):
#   for j in range(len(weekDataK[i])):
#     tmp = weekDataK[0][j]
#     print(tmp.date+" "+tmp.time)

for i in range(totalWeek):
  dataAmount.append(len(weekDataK[i]))
  bbAmount.append(0)
  rsiAmount.append(0)

# calculate bollinger band
for i in range(totalWeek):
  each = weekDataK[i]
  for j in range(len(each)):
    if j>=bbVal-1:
      curSum = 0
      for k in range(j-bbVal+1,j): # only 19 in front
        curSum+=float(each[k].close)
      curAvr = curSum/(bbVal-1) 
      curSD = 0
      for k in range(j-bbVal+1,j): 
        curSD+=(curAvr-float(each[k].close))*(curAvr-float(each[k].close))
      curSD/=(bbVal-1)
      curSD = math.sqrt(curSD)
      curBB = eachBB(curAvr,curAvr+sdVal*curSD,curAvr-sdVal*curSD)
      # print(each[k].close,end=" ")
      # print(curAvr,end=" ")
      # print(curAvr+sdVal*curSD,end=" ")
      # print(curAvr-sdVal*curSD)
      bbDataK[i].append(curBB)
      bbAmount[i]+=1
    else:
      bbDataK[i].append(eachBB(-1,-1,-1))
    
# calculate rsi value
for i in range(totalWeek):
  each = weekDataK[i]
  for j in range(len(each)):
    if j>=rsiVal-1:
      avrGain = 0
      avrLoss = 0
      for k in range(j-rsiVal+1,j+1): # all 14 including the current one
        change = each[k].close-each[k].open
        if change>0:
          avrGain+=change
        else:
          avrLoss-=change
      avrGain/=rsiVal
      avrLoss/=rsiVal
      if avrLoss!=0:
        rs = avrGain/avrLoss
        rsi = 100-(100/(1+rs))
      else:
        rsi = 100
      rsiDataK[i].append(rsi)
      rsiAmount[i]+=1
    else:
      rsiDataK[i].append(-1)

# create idx of each data bb rsi
for i in range(totalWeek):
  data = weekDataK[i]
  bb = bbDataK[i]
  rsi = rsiDataK[i]
  dataIdx.append(range(dataAmount[i]))
  bbDataIdx.append(range(dataAmount[i]-bbAmount[i],dataAmount[i]))
  rsiDataIdx.append(range(dataAmount[i]-rsiAmount[i],dataAmount[i]))
  avaDataIdx.append(range(max(dataAmount[i]-bbAmount[i],dataAmount[i]-rsiAmount[i]),dataAmount[i]))
  # plot(i)

# creating and evaluating signal
for i in range(totalWeek):
  signal.append([])
  for j in range(dataAmount[i]):
    signal[i].append(0)
  prevBbSig = -5
  prevRsiSig = -5
  curSideway = 0
  curSuccess = 0
  for j in avaDataIdx[i]:
    if weekDataK[i][j].close>=bbDataK[i][j].upper:
      bbSig = -1
    elif weekDataK[i][j].close<=bbDataK[i][j].lower:
      bbSig = 1
    else:
      bbSig = 0
    if rsiDataK[i][j]>=rsiUpperLim:
      rsiSig = -1
    elif rsiDataK[i][j]<=rsiLowerLim:
      rsiSig = 1
    else:
      rsiSig = 0
    curSig = 0
    if bbSig==rsiSig and bbSig!=0:
      if bbSig!=prevBbSig or rsiSig!=prevRsiSig:
        curSig = bbSig
        curSideway = 0
        curSuccess = 0
        sigAll+=1
      else:
        curSideway+=1
        if curSideway<=martingale and curSuccess==0:
          curSig = bbSig
    signal[i][j] = curSig
    prevBbSig = bbSig
    prevRsiSig = rsiSig
    if curSig!=0 and j<dataAmount[i]-1:
      curStatus = -1
      if signal[i][j]==-1 and weekDataK[i][j].close>weekDataK[i][j+1].close:
        curStatus = 1
      if signal[i][j]==1 and weekDataK[i][j].close<weekDataK[i][j+1].close:
        curStatus = 1
      if weekDataK[i][j].close==weekDataK[i][j+1].close:
        curStatus = 0
      if curStatus==0:
        curSideway-=1
      elif curStatus==1:
        curSuccess = 1
        sigSuccess+=1
  #   print(cursig,end = " ")
  # print()

sigPercent = sigSuccess/sigAll*100
file = open("/content/drive/My Drive/Colab Notebooks/result.csv","w")
bbP = str(bbVal)
rsiP = str(rsiVal)
sigP = str(sigPercent)
file.write(bbP+","+rsiP+","+sigP)
file.close()

#print(sigPercent)
    


In [70]:
bbLo = 10
bbHi = 30
sdVal = 2
rsiLo = 8
rsiHi = 20
width = 5
rsiUpperLim = 70
rsiLowerLim = 30
martingale = 1

def printToFile(bbVal,sdVal,rsiVal,width,rsiUpperLim,rsiLowerLim,martingale):
  bbVal = str(bbVal)
  sdVal = str(sdVal)
  rsiVal = str(rsiVal)
  width = str(width)
  rsiUpperLim = str(rsiUpperLim)
  rsiLowerLim = str(rsiLowerLim)
  martingale = str(martingale)
  filepath = "/content/drive/My Drive/Colab Notebooks/parameter.csv"
  file = open(filepath,"w")
  file.write(bbVal+","+sdVal+","+rsiVal+","+width+","+rsiUpperLim+","+rsiLowerLim+","+martingale)
  file.close()

file = open("/content/drive/My Drive/Colab Notebooks/forex.csv","w")
file.write("EURJPY,202204")
file.close()

bestSigPercent = -1

for bbVal in range(bbLo,bbHi+1):
  for rsiVal in range(rsiLo,rsiHi+1):
    printToFile(bbVal,sdVal,rsiVal,width,rsiUpperLim,rsiLowerLim,martingale)
    print("BB:",end = "")
    print(bbVal,end = " ")
    print("RSI:",end = "")
    if rsiVal<10:
      print(" ",end="")
    print(rsiVal,end = "")
    print("; percent accuracy : ",end = "")
    exec(open("/content/drive/My Drive/Colab Notebooks/run.py").read())
    file = open("/content/drive/My Drive/Colab Notebooks/result.csv")
    csvreader = csv.reader(file)
    for x in csvreader:
      sigPercent = float(x[2])
    print(sigPercent)
    if sigPercent>bestSigPercent:
      bestSigPercent = sigPercent
      bestBB = bbVal
      bestRSI = rsiVal

print()
print("best parameter config is")
print("BB:",end = "")
print(bestBB,end = " ")
print("RSI:",end = "")
if bestRSI<10:
  print(" ",end="")
print(bestRSI,end = "")
print("; percent accuracy : ",end = "")
print(bestSigPercent)
    


BB:10 RSI: 8; percent accuracy : 74.06639004149378
BB:10 RSI: 9; percent accuracy : 73.47417840375586
BB:10 RSI:10; percent accuracy : 71.64556962025317
BB:10 RSI:11; percent accuracy : 73.92550143266476
BB:10 RSI:12; percent accuracy : 73.4567901234568
BB:10 RSI:13; percent accuracy : 71.72413793103448
BB:10 RSI:14; percent accuracy : 72.47386759581882
BB:10 RSI:15; percent accuracy : 77.32342007434944
BB:10 RSI:16; percent accuracy : 76.98412698412699
BB:10 RSI:17; percent accuracy : 78.57142857142857
BB:10 RSI:18; percent accuracy : 75.25773195876289
BB:10 RSI:19; percent accuracy : 75.0
BB:10 RSI:20; percent accuracy : 76.33136094674556
BB:11 RSI: 8; percent accuracy : 76.80851063829788
BB:11 RSI: 9; percent accuracy : 76.16822429906543
BB:11 RSI:10; percent accuracy : 74.35897435897436
BB:11 RSI:11; percent accuracy : 76.61971830985915
BB:11 RSI:12; percent accuracy : 75.15527950310559
BB:11 RSI:13; percent accuracy : 73.10344827586206
BB:11 RSI:14; percent accuracy : 74.564459930